In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("../resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_model_snr']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
data = selected_features
target = df["koi_disposition"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ss,koi_model_snr
6122,0,0,0,10.8
6370,0,0,1,13.8
2879,1,0,0,254.3
107,0,0,0,38.4
29,0,0,0,696.5


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [9]:
#Create SVM Model
from sklearn.svm import SVC

svc_m = SVC(kernel='linear')
svc_m.fit(X_train_scaled, y_train)


SVC(kernel='linear')

In [10]:
print(f"Training Data Score: {svc_m.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {svc_m.score(X_test_scaled, y_test)}")

Training Data Score: 0.7379362960137326
Testing Data Score: 0.7437070938215103


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [11]:
# Create the GridSearchCV model
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 1, 3, 4, 5, 6, 7, 8, 10, 50, 100],
    'gamma': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 5, 10]}

grid_svc = GridSearchCV(svc_m, param_grid, verbose=3)

In [12]:
# Train the model with GridSearch
grid_svc.fit(X_train_scaled, y_train)

] END ..................................C=4, gamma=10; total time=   0.2s
[CV 5/5] END ..................................C=4, gamma=10; total time=   0.2s
[CV 1/5] END ...............................C=5, gamma=1e-06; total time=   0.2s
[CV 2/5] END ...............................C=5, gamma=1e-06; total time=   0.3s
[CV 3/5] END ...............................C=5, gamma=1e-06; total time=   0.3s
[CV 4/5] END ...............................C=5, gamma=1e-06; total time=   0.3s
[CV 5/5] END ...............................C=5, gamma=1e-06; total time=   0.2s
[CV 1/5] END ...............................C=5, gamma=1e-05; total time=   0.3s
[CV 2/5] END ...............................C=5, gamma=1e-05; total time=   0.3s
[CV 3/5] END ...............................C=5, gamma=1e-05; total time=   0.2s
[CV 4/5] END ...............................C=5, gamma=1e-05; total time=   0.2s
[CV 5/5] END ...............................C=5, gamma=1e-05; total time=   0.2s
[CV 1/5] END ......................

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 3, 4, 5, 6, 7, 8, 10, 50, 100],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 5, 10]},
             verbose=3)

In [13]:
print(grid_svc.best_params_)
print(grid_svc.best_score_)

{'C': 5, 'gamma': 1e-06}
0.7547176882381621


# Save the Model

In [14]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'svc_model.sav'
joblib.dump(grid_svc, filename)

['svc_model.sav']